In [12]:
import re

In [1]:
import adagram
from lxml import html
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.corpus import wordnet as wn
from pymorphy2 import MorphAnalyzer
from string import punctuation
import json, os, re, sys
from collections import Counter
import numpy as np
from razdel import tokenize as razdel_tokenize
from matplotlib import pyplot as plt
import gensim
import pandas as pd
from sklearn.metrics import adjusted_rand_score
import tqdm
from sklearn.metrics import f1_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.model_selection import KFold


%matplotlib inline
import warnings
warnings.filterwarnings('ignore')
morph = MorphAnalyzer()
punct = punctuation+'«»—…“”*№–'
stops = set(stopwords.words('russian'))

def normalize(text):
    
    words = [token.text.strip(punct) for token in list(razdel_tokenize(text))]
    words = [morph.parse(word)[0].normal_form for word in words if word and word not in stops]

    return words


In [4]:
# запустите если не установлен ворднет
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\ruthenian8\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\wordnet.zip.


True

### Домашнее задание. Реализовать алгоритм Леска и проверить его на реальном датасете

In [27]:
def lesk( word, sentence ):
    """Ваш код тут"""
    bestsense = 0
    maxoverlap = 0 
    orig_set = set(sentence)
    wn_sets = []
    for i, syns in enumerate(wn.synsets(word)):
        wn_sets.append((i,
                        set(re.findall(r"[a-zA-Z]+", syns.definition()))))
         
    wn_sorted = sorted(wn_sets,
                       key=lambda x: len(x[1].intersection(orig_set)),
                       reverse=True)
    bestsense = wn_sorted[0][0]
    return bestsense


In [35]:
class leskMeasure():
    def __init__(self, subcorpus):
        self.subcorpus = subcorpus
        self.all = 0
        for sent in subcorpus:
            for word in sent:
                if word[0] != "":
                    self.all += 1
                    
    @staticmethod
    def lesk( word, sentence ):
        """Ваш код тут"""
        bestsense = 0
        maxoverlap = 0 
        orig_set = set(sentence)
        wn_sets = []
        for i, syns in enumerate(wn.synsets(word)):
            wn_sets.append((i,
                            set(re.findall(r"[a-zA-Z]+", syns.definition()))))

        wn_sorted = sorted(wn_sets,
                           key=lambda x: len(x[1].intersection(orig_set)),
                           reverse=True)
        bestsense = wn_sorted[0][0]
        return bestsense                    
                    
    @staticmethod
    def process_sentence(sent):
        trues = 0
        full_sentence = [x[1] for x in sent] 
        for item in sent:
            if item[0] == "":
                continue
            trues += leskMeasure.process_word(item, full_sentence)
        return trues
    
    @staticmethod
    def process_word(item, sent):
        if wn.synsets(item[1])[leskMeasure.lesk(item[1], sent)] == \
        wn.lemma_from_key(item[0]).synset():
            return 1
        return 0
    
    def compute_accuracy(self):
        true_counter = 0
        for sent in self.subcorpus:
            sent_true_count = self.process_sentence(sent)
            true_counter += sent_true_count
        return true_counter / self.all

In [10]:
corpus_wsd = []
corpus = open('corpus_wsd_50k.txt').read().split('\n\n')
for sent in corpus:
    corpus_wsd.append([s.split('\t') for s in sent.split('\n')])

In [36]:
subcorpus = corpus_wsd[:1000]
that = leskMeasure(subcorpus)
that.compute_accuracy()

0.36496771217712176

In [37]:
subcorpus = corpus_wsd[1000:3000]
that = leskMeasure(subcorpus)
that.compute_accuracy()

0.37930832356389216

In [38]:
subcorpus = corpus_wsd[3000:5000]
that = leskMeasure(subcorpus)
that.compute_accuracy()

0.3604372722540344